In [1]:
# Imports
from biocrnpyler import *
from genelet import *
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

## Cell 1 simple rR1 transport w/ rR1 external reservoir

In [2]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/rR1_external_reservoir.xml")

# Create a simple rR1 membrane 
mb1 = createSubsystem("SBML_Models/simplemembrane_rR1.xml", membrane = True)

cell_1 = System("cell_1")
cell_1.setInternal([ss1])
cell_1.setExternal([ss2])
cell_1.setMembrane(mb1)
cell_1_model = cell_1.getModel()


#cell_1 = System ("cell_1", ListOfInternalSubsystems = [ss1], 
#                 ListOfExternalSubsystems = [ss2], 
#                 ListOfMembraneSubsystems = [mb1])

##### Set Species Amount #####
cell_1_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'cell_1_external')
#cell_1_model.setSpeciesAmount('rna_rR1', 100, compartment = 'cell_1_external')

cell_1_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('Core2_OFF', 4e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAP', 300, compartment = 'cell_1_internal')
#cell_1_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_1_model.writeSBML('cell_1_model.xml')


The subsystem from SBML_Models/simplemembrane_rR1.xml has multiple compartments


1

In [3]:
# Calling Names

X_id1 = cell_1_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_1_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_1_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_1_model.getSpeciesByName('Core2_OFF').getId()

X_id5 = cell_1_model.getSpeciesByName("rna_rR2").getId()

X_id6 = cell_1_model.getSpeciesByName("rna_rR1", compartment = 'cell_1_internal').getId()
X_id7 = cell_1_model.getSpeciesByName("rna_rR1_e", compartment = 'cell_1_external').getId()

X_id8 = cell_1_model.getSpeciesByName("complex_Core2_AI").getId()
X_id9 = cell_1_model.getSpeciesByName("complex_Core1_AI").getId()

X_id10 = cell_1_model.getSpeciesByName("dna_dA2").getId()
X_id11 = cell_1_model.getSpeciesByName("dna_dA1").getId()

print (X_id6)

rna_rR1_1_combined


In [4]:
# Simulate with BioScrape
timepoints = np.linspace(0, 28800, 1000)
results_1,_ = cell_1_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, results_1[X_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, results_1[X_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
#b.circle(timepoints, results_1[X_id5], legend_label = "rR2", color = "red")
b.circle(timepoints, results_1[X_id6], legend_label = "rR1_internal", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, results_1[X_id7], legend_label = "rR1_external", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

bokeh.io.show(row(a, b, c))
warnings.filterwarnings("ignore")

C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:93: UserWarning: SBML model contains reversible reaction!
Please check rate expressions and ensure they are non-negative before doing stochastic simulations.
  'Please check rate expressions and ensure they are non-negative before doing '+
C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:208: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
C:\Users\Jeremiah\anaconda3\lib\site-packages\scipy\integrate\odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

Loading BokehJS ...

In [5]:
d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, results_1[X_id8], legend_label = "rR1_i:dA1", color = "orange")
d.circle(timepoints, results_1[X_id9], legend_label = "rR2:dA2", color = "magenta")
d.legend.click_policy="hide"
d.legend.location="bottom_right"


f = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
f.circle(timepoints, results_1[X_id10], legend_label = "dA2", color = "magenta")
f.circle(timepoints, results_1[X_id11], legend_label = "dA1", color = "orange")
f.legend.click_policy="hide"
f.legend.location="top_right"

X_id12 = cell_1_model.getSpeciesByName("complex_Core1_ON_protein_RNAP").getId()
X_id13 = cell_1_model.getSpeciesByName("complex_Core2_ON_protein_RNAP").getId()

g = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
g.circle(timepoints, results_1[X_id12], legend_label = "Core1_ON_RNAP", color = "blue")
#g.circle(timepoints, results_1[X_id13], legend_label = "Core2_ON_RNAP", color = "red")
g.legend.click_policy="hide"
g.legend.location="center_right"



bokeh.io.show(row(d, f, g))
warnings.filterwarnings("ignore")

## Cell 2 with transport protein ##

In [14]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/rR1_external_reservoir.xml")

# Create a simple rR1 membrane 
mb2 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)

cell_2 = System("cell_2")
cell_2.setInternal([ss1])
cell_2.setExternal([ss2])
cell_2.setMembrane(mb2)

cell_2_model = cell_2.getModel()

##### Set Species Amount #####

#cell_2_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'cell_2_external')
cell_2_model.setSpeciesAmount('rna_rR1_e', 100, compartment = 'cell_2_external')

cell_2_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('Core2_OFF', 5e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_2_internal')

cell_2_model.setSpeciesAmount('protein_RNAseH', 20, compartment = 'cell_2_internal')
#cell_2_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_2_internal')

cell_2_model.setSpeciesAmount('protein_RNAP', 150, compartment = 'cell_2_internal')

#cell_2_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_2_model.writeSBML('SBML_Models/cell_2_model.xml')

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


1

In [15]:
# Calling Names

X_id1 = cell_2_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_2_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_2_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_2_model.getSpeciesByName('Core2_OFF').getId()

X_id5 = cell_2_model.getSpeciesByName("rna_rR2").getId()
X_id6 = cell_2_model.getSpeciesByName("rna_rR1", compartment = 'cell_2_internal').getId()
X_id7 = cell_2_model.getSpeciesByName("rna_rR1_e", compartment = 'cell_2_external').getId()

X_id8 = cell_2_model.getSpeciesByName("complex_Core2_AI").getId()
X_id9 = cell_2_model.getSpeciesByName("complex_Core1_AI").getId()

X_id10 = cell_2_model.getSpeciesByName("dna_dA2").getId()
X_id11 = cell_2_model.getSpeciesByName("dna_dA1").getId()

In [16]:
# Simulate with BioScrape
timepoints = np.linspace(0, 28800, 1000)
results_2,_ = cell_2_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, results_2[X_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, results_2[X_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, results_2[X_id5], legend_label = "rR2" , color = "red")
b.circle(timepoints, results_2[X_id6], legend_label = "rR1_i", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, results_2[X_id7], legend_label = "rR1_e", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

bokeh.io.show(row(a, c, b))

odeint failed with mxstep=500...

Loading BokehJS ...

In [17]:
d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, results_2[X_id8], legend_label = "rR1_i:dA1", color = "orange")
d.circle(timepoints, results_2[X_id9], legend_label = "rR2:dA2", color = "magenta")
d.legend.click_policy="hide"
d.legend.location="bottom_right"


f = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
f.circle(timepoints, results_2[X_id10], legend_label = "dA2", color = "magenta")
f.circle(timepoints, results_2[X_id11], legend_label = "dA1", color = "orange")
f.legend.click_policy="hide"
f.legend.location="top_right"



bokeh.io.show(row(d, f))
warnings.filterwarnings("ignore")